In [1]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [2]:
train = pd.read_csv('../input/train.csv')
print(train.shape)
train.head()

(1817, 5)


,image_id,healthy,multiple_diseases,rust,scab
0,Train_0,0,0,0,1
1,Train_1,0,1,0,0
2,Train_2,1,0,0,0
3,Train_3,0,0,1,0
4,Train_4,1,0,0,0
